In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
from urllib.parse import urljoin, urlparse
from typing import List, Dict, Optional, Tuple
import re

from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
import csv
import ast

import csv
from io import StringIO, BytesIO
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
from urllib.parse import urlparse
import csv
from io import StringIO, BytesIO
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
from datetime import datetime

In [2]:
BASE = "https://www.betexplorer.com"

start_sort = 9
ignore = 5

In [3]:

def download(pk):
    def get_first_fixture_block(league_href):
        fixtures_url = urljoin(BASE, league_href.rstrip("/") + "/fixtures/")
        html = requests.get(fixtures_url, timeout=15).text
        soup = BeautifulSoup(html, "lxml")

        table = soup.select_one("#js-leaguefixtures-all table.table-main")
        if not table:
            return []

        rows = table.find_all("tr")

        blocks = []          # will hold [block1, block2]
        current_block = []   # temp holder for matches
        started = False      # once we meet first round header

        for tr in rows:
            # Detect header row ("12. Round", "13. Round", etc)
            if tr.find("th"):
                if not started:
                    # First header found → start first block
                    started = True
                    # start new block
                    if current_block:
                        blocks.append(current_block)
                    current_block = []
                    continue
                else:
                    # header after starting → new block
                    if current_block:
                        blocks.append(current_block)

                    if len(blocks) == 2:
                        break  # we only care about 2 blocks

                    current_block = []
                    continue

            if not started:
                continue

            # extract datetime
            dt_td = tr.find("td", class_="table-main__datetime")
            if not dt_td:
                continue
            datetime_text = dt_td.get_text(strip=True)

            # extract match link
            link_td = tr.find("a", class_="in-match")
            if not link_td:
                continue
            match_href = link_td["href"]
            match_url = urljoin(BASE, match_href)

            # extract teams
            spans = link_td.find_all("span")
            if len(spans) != 2:
                continue
            home = spans[0].get_text(strip=True)
            away = spans[1].get_text(strip=True)

            current_block.append({
                "datetime": datetime_text,
                "home": home,
                "away": away,
                "match_link": match_url
            })

        # append last collected block
        if current_block:
            blocks.append(current_block)

        # ---- DECISION LOGIC ----
        if not blocks:
            return []

        first_block = blocks[0] if len(blocks) > 0 else []
        second_block = blocks[1] if len(blocks) > 1 else []

        if len(first_block) > ignore:
            return first_block  # return first block only
        else:
            return second_block  # return second block instead


    leagues = [
        {"name": 'League', "href": pk},
    ]

    linkss = []

    for lg in leagues:
        block = get_first_fixture_block(lg["href"])
        for m in block:
            linkss.append({
            "date": m["datetime"],
            "home_team": m["home"],
            "away_team": m["away"],
            "link": m["match_link"]
        })


    def compute_fixture_stats(home_team, away_team, home_results, away_results, date):
        """
        home_results and away_results must already be:
        - filtered to same competition
        - sorted from most recent → oldest
        """

        def extract_home_count(matches):
            return sum(1 for m in matches if m["is_home"] is True)

        def extract_away_count(matches):
            return sum(1 for m in matches if m["is_home"] is False)

        # -------------------------------
        # TRY LAST 6 MATCHES (STRICT 3-3)
        # -------------------------------
        if len(home_results) >= start_sort and len(away_results) >= start_sort:
            last6_home = home_results[:start_sort]
            last6_away = away_results[:start_sort]

            H_home_count = extract_home_count(last6_home)
            A_away_count = extract_away_count(last6_away)

            if H_home_count == 5 and A_away_count == 5:
                sorting_balance = start_sort
                home_selected = [m for m in last6_home if m["is_home"]]
                away_selected = [m for m in last6_away if not m["is_home"]]
            else:
                sorting_balance = None
                home_selected = None
                away_selected = None
        else:
            sorting_balance = None
            home_selected = None
            away_selected = None

        # --------------------------------------
        # TRY LAST 7 MATCHES IF LAST 6 FAILED
        # --------------------------------------
        if sorting_balance is None:
            next_sort = start_sort + 1
            if len(home_results) >= next_sort and len(away_results) >= next_sort:
                last7_home = home_results[:next_sort]
                last7_away = away_results[:next_sort]

                H_count = extract_home_count(last7_home)
                A_count = extract_away_count(last7_away)

                # STRICT RULE: must be equal AND must be 3 or 4
                if H_count == A_count and H_count in (5, 5):
                    sorting_balance = next_sort
                    home_selected = [m for m in last7_home if m["is_home"]]
                    away_selected = [m for m in last7_away if not m["is_home"]]
                else:
                    return None  # reject completely
            else:
                return None

        # At this point sorting balance is validated strictly.

        # ------------------ COMPUTATION HELPERS ------------------

        def avg_goals(matches):
            gf = sum(m["goals_for"] for m in matches) / len(matches)
            ga = sum(m["goals_against"] for m in matches) / len(matches)
            return gf, ga

        def count_scored_2plus(matches):
            return sum(1 for m in matches if m["goals_for"] >= 2) / len(matches)

        def count_conceded_2plus(matches):
            return sum(1 for m in matches if m["goals_against"] >= 2) / len(matches)

        def outcomes(matches):
            wins = sum(1 for m in matches if m["goals_for"] > m["goals_against"])
            draws = sum(1 for m in matches if m["goals_for"] == m["goals_against"])
            losses = sum(1 for m in matches if m["goals_for"] < m["goals_against"])
            total = len(matches)
            return wins/total, draws/total, losses/total

        # ------------------ HOME TEAM STATS ------------------
#         HTS, HTC = avg_goals(home_selected)
        HS_plus = count_scored_2plus(home_selected)
        HC_plus = count_conceded_2plus(home_selected)
        PHW, PHX, PHL = outcomes(home_selected)

        # ------------------ AWAY TEAM STATS ------------------
#         ATS, ATC = avg_goals(away_selected)
        AS_plus = count_scored_2plus(away_selected)
        AC_plus = count_conceded_2plus(away_selected)
        PAW, PAX, PAL = outcomes(away_selected)
        
        # Home Scored
        HS1 = home_results[0]['goals_for']
        HS2 = home_results[1]['goals_for']
        HS3 = home_results[2]['goals_for']
        HS4 = home_results[3]['goals_for']
        HS5 = home_results[4]['goals_for']

        # Home Conceded
        HC1 = home_results[0]['goals_against']
        HC2 = home_results[1]['goals_against']
        HC3 = home_results[2]['goals_against']
        HC4 = home_results[3]['goals_against']
        HC5 = home_results[4]['goals_against']

        # Away Scored
        AS1 = away_results[0]['goals_for']
        AS2 = away_results[1]['goals_for']
        AS3 = away_results[2]['goals_for']
        AS4 = away_results[3]['goals_for']
        AS5 = away_results[4]['goals_for']

        # Away Conceded
        AC1 = away_results[0]['goals_against']
        AC2 = away_results[1]['goals_against']
        AC3 = away_results[2]['goals_against']
        AC4 = away_results[3]['goals_against']
        AC5 = away_results[4]['goals_against']
        
        HTS_new = (HS1 + HS2 + HS3 + HS4 + HS5) / 5
        HTC_new = (HC1 + HC2 + HC3 + HC4 + HC5) / 5
        ATS_new = (AS1 + AS2 + AS3 + AS4 + AS5) / 5
        ATC_new = (AC1 + AC2 + AC3 + AC4 + AC5) / 5
        

        return {
                "sorting_balance": sorting_balance,
                "home_team": home_team,
                "away_team": away_team,
                "date": date,
                "stats": {
                    # Existing averages
#                     "HTS": HTS, "HTC": HTC,
#                     "ATS": ATS, "ATC": ATC,

                    # Individual Home Scored
                    "HS1": HS1, "HS2": HS2, "HS3": HS3, "HS4": HS4, "HS5": HS5,

                    # Individual Home Conceded
                    "HC1": HC1, "HC2": HC2, "HC3": HC3, "HC4": HC4, "HC5": HC5,

                    # Individual Away Scored
                    "AS1": AS1, "AS2": AS2, "AS3": AS3, "AS4": AS4, "AS5": AS5,

                    # Individual Away Conceded
                    "AC1": AC1, "AC2": AC2, "AC3": AC3, "AC4": AC4, "AC5": AC5,

                    # New rolling averages (last 5)
#                     "HTS_new": HTS_new,
#                     "HTC_new": HTC_new,
#                     "ATS_new": ATS_new,
#                     "ATC_new": ATC_new,

                    # Probability metrics
                    "HS_plus_1": HS_plus,
                    "HS-1": 1 - HS_plus,

                    "HC_plus_1": HC_plus,
                    "HC-1": 1 - HC_plus,

                    "AS_plus_1": AS_plus,
                    "AS-1": 1 - AS_plus,

                    "AC_plus_1": AC_plus,
                    "AC-1": 1 - AC_plus,

                    # Match outcome probabilities
                    "PHW": PHW, "PHX": PHX, "PHL": PHL,
                    "PAW": PAW, "PAX": PAX, "PAL": PAL
                }
            }






    # -----------------------------------------------------------
    # 1) Extract team links and competition link from match page
    # -----------------------------------------------------------
    def get_team_links_and_comp(match_url):

        html = requests.get(match_url, timeout=15).text
        soup = BeautifulSoup(html, "lxml")

        # team links are the first two <a> inside .list-details
        team_tags = soup.select("ul.list-details li.list-details__item a")

        if len(team_tags) < 2:
            raise ValueError("Could not find 2 team links in match page")

        home_href = team_tags[0]["href"]
        away_href = team_tags[1]["href"]

        home_url = urljoin(BASE, home_href)
        away_url = urljoin(BASE, away_href)

        # competition shown in header breadcrumb
        comp_tag = soup.select_one("ul.breadcrumb__ul li:nth-child(4) a")
        if not comp_tag:
            raise ValueError("Could not detect competition link")

        comp_href = comp_tag["href"]  # e.g. /football/england/premier-league/

        return home_url, away_url, comp_href


    # -----------------------------------------------------------
    # 2) Fetch team /results/ page
    # -----------------------------------------------------------
    def get_team_results_page(team_url):
        results_url = team_url.rstrip("/") + "/results/"

        html = requests.get(results_url, timeout=15).text
        soup = BeautifulSoup(html, "lxml")
        return soup


    # -----------------------------------------------------------
    # 3) Extract rows belonging ONLY to this competition
    # -----------------------------------------------------------
    def get_results_for_competition(soup, competition_href):
        results = []
        in_target_comp = False

        # normalize competition
        competition_href = competition_href.rstrip("/")

        # each block starts with <th class="table-main__tournament">
        for tr in soup.select("table.teaminfo tr"):

            # --- COMPETITION HEADER DETECTION ---
            header = tr.find("a", class_="table-main__tournament")
            if header:
                comp_slug = header["href"].rstrip("/")
                # check if this is the correct league
                in_target_comp = (competition_href in comp_slug)
                continue

            if not in_target_comp:
                continue

            # --- NORMAL MATCH ROW ---
            cols = tr.find_all("td")
            if len(cols) < 5:
                continue

            # find score td
            score_td = None
            for td in cols:
                if ":" in td.get_text():
                    score_td = td
                    break

            if not score_td:
                continue

            # extract score
            m = re.search(r"(\d+)\s*:\s*(\d+)", score_td.get_text())
            if not m:
                continue

            # determine home/away for THIS team
            # The <strong> tag marks the current team
            home_col = cols[2]
            away_col = cols[3]

            is_home = home_col.find("strong") is not None

            home_goals = int(m.group(1))
            away_goals = int(m.group(2))

            if is_home:
                gf, ga = home_goals, away_goals
                opponent = away_col.get_text(strip=True)
            else:
                gf, ga = away_goals, home_goals
                opponent = home_col.get_text(strip=True)

            results.append({
                "round": cols[1].get_text(strip=True),
                "is_home": is_home,
                "opponent": opponent,
                "goals_for": gf,
                "goals_against": ga,
                "date": cols[-1].get_text(strip=True),
            })

        return results


    # -----------------------------------------------------------
    # 4) RUN FULL FLOW
    # -----------------------------------------------------------

    final_list = []
    for fixture_url in linkss:
        
        home_url, away_url, competition_href = get_team_links_and_comp(fixture_url["link"])

        # fetch /results/ pages
        home_soup = get_team_results_page(home_url)
        away_soup = get_team_results_page(away_url)

        # extract filtered results
        home_results = get_results_for_competition(home_soup, competition_href)
        away_results = get_results_for_competition(away_soup, competition_href)

        homes = []
        aways = []

        for r in home_results:
            homes.append(r)

        for r in away_results:
            aways.append(r)

        # append this fixture's data to the master list
        final_list.append({
            "fixture_url": fixture_url["link"],
            "home_url": home_url,
            "away_url": away_url,
            "homes": homes,
            "aways": aways,
            "date": fixture_url["date"]
        })
    

    def url_split(url):
        slug = url.split("/team/")[1].split("/")[0]
        team_name = slug.replace("-", " ").title()
        return team_name

    final_output = []
    for item in final_list:
        stats = compute_fixture_stats(
            url_split(item["home_url"]),
            url_split(item["away_url"]),
            item["homes"],
            item["aways"],
            item["date"],
        )

        final_output.append(stats)
        

    def save_output_to_files_template(final_output, csv_name="fixture_stats.csv", excel_name="fixture_stats.xlsx"):
        """
        Saves final_output to CSV and Excel following the EXACT template order requested.
        """

        # -----------------------------------------------------------
        # 1. FIXED TEMPLATE COLUMN ORDER (DO NOT CHANGE)
        # -----------------------------------------------------------
        columns = [
            "sorting_balance",
            "home_team",
            "away_team",
            "date",

            "HS1", "HS2", "HS3", "HS4", "HS5",
            "HC1", "HC2", "HC3", "HC4", "HC5",
            "AS1", "AS2", "AS3", "AS4", "AS5",
            "AC1", "AC2", "AC3", "AC4", "AC5"

            "HS_plus_1", "HS-1",
            "HC_plus_1", "HC-1",
            "AS_plus_1", "AS-1",
            "AC_plus_1", "AC-1",

            "PHW", "PHX", "PHL",
            "PAW", "PAX", "PAL",
        ]

        # -----------------------------------------------------------
        # 2. FLATTEN final_output
        # -----------------------------------------------------------
        rows = []
        for item in final_output:

            stats = item.get("stats", {})

            row = {
                "sorting_balance": item.get("sorting_balance"),
                "home_team": item.get("home_team"),
                "away_team": item.get("away_team"),
                "date": item.get("date"),

#                 "HTS": stats.get("HTS"),
#                 "HTC": stats.get("HTC"),
#                 "ATS": stats.get("ATS"),
#                 "ATC": stats.get("ATC"),
                "HS1": stats.get("HS1"), "HS2": stats.get("HS2"), "HS3": stats.get("HS3"), "HS4": stats.get("HS4"), "HS5": stats.get("HS5"),

                # Individual Home Conceded
                "HC1": stats.get("HC1"), "HC2": stats.get("HC2"), "HC3": stats.get("HC3"), "HC4": stats.get("HC4"), "HC5": stats.get("HC5"),

                # Individual Away Scored
                "AS1": stats.get("AS1"), "AS2": stats.get("AS2"), "AS3": stats.get("AS3"), "AS4": stats.get("AS4"), "AS5": stats.get("AS5"),

                # Individual Away Conceded
                "AC1": stats.get("AC1"), "AC2": stats.get("AC2"), "AC3": stats.get("AC3"), "AC4": stats.get("AC4"), "AC5": stats.get("AC5"),

                "HS_plus_1": stats.get("HS_plus_1"),
                "HS-1": stats.get("HS-1"),
                "HC_plus_1": stats.get("HC_plus_1"),
                "HC-1": stats.get("HC-1"),
                "AS_plus_1": stats.get("AS_plus_1"),
                "AS-1": stats.get("AS-1"),
                "AC_plus_1": stats.get("AC_plus_1"),
                "AC-1": stats.get("AC-1"),

                "PHW": stats.get("PHW"),
                "PHX": stats.get("PHX"),
                "PHL": stats.get("PHL"),
                "PAW": stats.get("PAW"),
                "PAX": stats.get("PAX"),
                "PAL": stats.get("PAL"),
            }

            rows.append(row)

        # -----------------------------------------------------------
        # 3. WRITE CSV (Simple)
        # -----------------------------------------------------------
        with open(csv_name, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()
            writer.writerows(rows)


        # -----------------------------------------------------------
        # 4. CREATE BEAUTIFUL EXCEL
        # -----------------------------------------------------------
        wb = Workbook()
        ws = wb.active

        # Title
        title = "MATCH FIXTURE STATISTICS"
        ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(columns))
        tcell = ws.cell(row=1, column=1)
        tcell.value = title
        tcell.font = Font(size=16, bold=True)
        tcell.alignment = Alignment(horizontal="center")
        tcell.fill = PatternFill(start_color="D9E1F2", fill_type="solid")

        # Header styling
        header_fill = PatternFill(start_color="BDD7EE", fill_type="solid")
        header_font = Font(bold=True)
        border = Border(
            left=Side(style="thin"),
            right=Side(style="thin"),
            top=Side(style="thin"),
            bottom=Side(style="thin")
        )

        # Header row
        for col_idx, col_name in enumerate(columns, start=1):
            cell = ws.cell(row=2, column=col_idx, value=col_name)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.border = border

        # Data rows with alternating colors
        fill_white = PatternFill(start_color="FFFFFF", fill_type="solid")
        fill_gray = PatternFill(start_color="F2F2F2", fill_type="solid")

        for row_idx, row in enumerate(rows, start=3):
            fill = fill_gray if row_idx % 2 == 0 else fill_white

            for col_idx, col_name in enumerate(columns, start=1):
                value = row.get(col_name, "")
                cell = ws.cell(row=row_idx, column=col_idx, value=value)
                cell.fill = fill
                cell.border = border

                if isinstance(value, (int, float)):
                    cell.alignment = Alignment(horizontal="center")
                else:
                    cell.alignment = Alignment(horizontal="left")

        # Auto-size columns
        for col_idx in range(1, len(columns) + 1):
            letter = get_column_letter(col_idx)
            max_length = max(len(str(ws.cell(row=r, column=col_idx).value or "")) for r in range(1, len(rows) + 3))
            ws.column_dimensions[letter].width = max_length + 2

        wb.save(excel_name)
        
    return final_output


def get_country_and_competition_from_url(url):
    """
    Given a URL like:
      https://www.betexplorer.com/football/england/premier-league/
    Returns:
      country = "England"
      competition = "Premier League"
    """
    parsed = urlparse(url)
    path = parsed.path  # e.g. "/football/england/premier-league/"
    parts = [p for p in path.split("/") if p]  # split and remove empty

    # parts should look like ["football", "england", "premier-league", ...]
    country = None
    competition = None

    # We assume the structure /football/<country>/<competition>/
    if len(parts) >= 3 and parts[0].lower() == "football":
        country = parts[1].replace("-", " ").title()
        competition = parts[2].replace("-", " ").title()
    else:
        # fallback: try last two parts
        if len(parts) >= 2:
            country = parts[-2].replace("-", " ").title()
            competition = parts[-1].replace("-", " ").title()

    return country, competition

In [4]:
def extract_all_league_links():
    url = BASE + "/"
    html = requests.get(url, timeout=15).text
    soup = BeautifulSoup(html, "lxml")

    leagues = []

    # Each league section looks like:
    # <ul class="leagues-list" data-country="england">
    league_boxes = soup.select("ul.leagues-list")

    for box in league_boxes:
        country = box.get("data-country", "").strip()

        # Each league row has this structure:
        # <a href="/football/england/premier-league/" ...>
        links = box.select("a.leaguesName500, a.leaguesName400, a.leaguesName350")

        for a in links:
            href = a.get("href", "").strip()

            if not href.startswith("/football/"):
                continue  # skip ads or invalid links

            # Extract league name (text inside <p>)
            p = a.find("p")
            if not p:
                continue

            full_name = p.get_text(strip=True)  # e.g. "England: Premier League"

            # Split into country and league
            if ":" in full_name:
                ctry, league = [x.strip() for x in full_name.split(":", 1)]
            else:
                ctry = country.capitalize()
                league = full_name

            # Construct full URL
            full_url = href

            leagues.append({
                "country": ctry,
                "league": league,
                "url": full_url
            })

    return leagues

linkings = []
# ------------------ TEST ------------------
if __name__ == "__main__":
    data = extract_all_league_links()
    for league in data:
        linkings.append(league['url'])

print(linkings)

['/football/nigeria/nnl/', '/football/africa/africa-cup-of-nations/', '/football/spain/laliga/', '/football/italy/serie-a/', '/football/germany/bundesliga/', '/football/africa/africa-cup-of-nations/', '/football/albania/albanian-cup/', '/football/angola/girabola/', '/football/asia/afc-asian-cup-u23/', '/football/australia/a-league/', '/football/australia/a-league-women/', '/football/bahrain/premier-league/', '/football/belgium/national-division-1-acff/', '/football/belgium/national-division-1-vv/', '/football/belgium/super-league-women/', '/football/bermuda/premier-league/', '/football/brazil/alagoano/', '/football/brazil/amazonense/', '/football/brazil/baiano/', '/football/brazil/brasiliense/', '/football/brazil/catarinense/', '/football/brazil/cearense/', '/football/brazil/gaucho/', '/football/brazil/goiano/', '/football/brazil/maranhense/', '/football/brazil/matogrossense/', '/football/brazil/mineiro/', '/football/brazil/paranaense/', '/football/brazil/paulista/', '/football/brazil/

In [6]:
linkings2 = ['/football/spain/laliga/', '/football/italy/serie-a/', 'football/germany/bundesliga/']

leagues = linkings2

found = []
count = 1
for i in leagues:
    print(f'Scrapping League Link {count} of {len(leagues)}...')
    country, competition = get_country_and_competition_from_url(i)
    leag = f'{country}-{competition}'
    found.append({'comps': download(i), 'league': leag})
    print(f'League Link {count} of {len(leagues)} Scraped!')
    count = count + 1
print('Everthing Scrapped Succesfully!')

Scrapping League Link 1 of 3...
League Link 1 of 3 Scraped!
Scrapping League Link 2 of 3...
League Link 2 of 3 Scraped!
Scrapping League Link 3 of 3...
League Link 3 of 3 Scraped!
Everthing Scrapped Succesfully!


In [9]:
cleaned = []

for block in found:   # your outer list
    comps = block.get("comps", [])
    cleaned_comps = [c for c in comps if c is not None]

    cleaned.append({
        "league": block.get("league"),
        "comps": cleaned_comps
    })

found2 = cleaned

In [10]:
excel_name = "computations"

In [11]:
def generate_files_from_output(final_output):
    """
    Returns IN-MEMORY CSV and EXCEL files 
    with each match taking 2 rows merged vertically,
    numeric data rounded to 2 decimals, and match info in a single wide cell.
    Adds a LEAGUE NAME column.
    """

    # Flatten dataset + attach league name
    flat_matches = []
    for block in final_output:
        league_name = block.get("league")
        comps = block.get("comps", [])
        for match in comps:
            match["league_name"] = league_name
            flat_matches.append(match)

    columns = [
        "sorting_balance",
        "match_info",
        "league_name",
#         "HTS", "HTC", "ATS", "ATC",
        "HS1", "HS2", "HS3", "HS4", "HS5",
        "HC1", "HC2", "HC3", "HC4", "HC5",
        "AS1", "AS2", "AS3", "AS4", "AS5",
        "AC1", "AC2", "AC3", "AC4", "AC5",
        "HS_plus_1", "HS-1", "HC_plus_1", "HC-1",
        "AS_plus_1", "AS-1", "AC_plus_1", "AC-1",
        "PHW", "PHX", "PHL", "PAW", "PAX", "PAL",
    ]

    rows = []

    for item in flat_matches:
        if item is None:
            continue

        stats = item.get("stats", {})

        match_info = f"{item.get('home_team','')}   {item.get('date','')}   {item.get('away_team','')}"

        row = {
            "sorting_balance": item.get("sorting_balance"),
            "match_info": match_info,
            "league_name": item.get("league_name"),
        }

        for key in ["HS1", "HS2", "HS3", "HS4", "HS5",
                    "HC1", "HC2", "HC3", "HC4", "HC5",
                    "AS1", "AS2", "AS3", "AS4", "AS5",
                    "AC1", "AC2", "AC3", "AC4", "AC5",
                    "HS_plus_1","HS-1","HC_plus_1","HC-1",
                    "AS_plus_1","AS-1","AC_plus_1","AC-1",
                    "PHW","PHX","PHL","PAW","PAX","PAL"]:
            val = stats.get(key)
            if val is not None:
                val = round(val, 2)
            row[key] = val

        rows.append(row)

    # ----------------- CSV IN MEMORY -----------------
    csv_buffer = StringIO()
    writer = csv.DictWriter(csv_buffer, fieldnames=columns)
    writer.writeheader()
    writer.writerows(rows)
    csv_data = csv_buffer.getvalue()

    # ----------------- EXCEL IN MEMORY -----------------
    excel_buffer = BytesIO()
    wb = Workbook()
    ws = wb.active

    # TITLE
    total_cols = len(columns)
    ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=total_cols)
    tcell = ws.cell(row=1, column=1, value="MATCH FIXTURE STATISTICS")
    tcell.font = Font(size=18, bold=True)
    tcell.alignment = Alignment(horizontal="center")
    tcell.fill = PatternFill(start_color="D9E1F2", fill_type="solid")

    # HEADER
    header_fill = PatternFill(start_color="BDD7EE", fill_type="solid")
    header_font = Font(bold=True)

    for col_idx, col_name in enumerate(columns, start=1):
        cell = ws.cell(row=2, column=col_idx, value=col_name)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal="center")

    # DATA (merge every two rows)
    start_row = 3
    for row in rows:
        for col_idx, col in enumerate(columns, start=1):
            ws.merge_cells(start_row=start_row, start_column=col_idx, end_row=start_row+1, end_column=col_idx)
            cell = ws.cell(row=start_row, column=col_idx, value=row.get(col))
            cell.alignment = Alignment(horizontal="center")
        start_row += 2

    # COLUMN WIDTHS
    for col_idx, col in enumerate(columns, start=1):
        letter = get_column_letter(col_idx)

        if col == "match_info":
            ws.column_dimensions[letter].width = 50
        elif col == "league_name":
            ws.column_dimensions[letter].width = 25
        else:
            ws.column_dimensions[letter].width = 18

    wb.save(excel_buffer)
    excel_buffer.seek(0)
    
    return csv_data, excel_buffer


csv_data, excel_buffer = generate_files_from_output(found2)

# Save CSV
# with open("fixtures.csv", "w", newline="", encoding="utf-8") as f:
#     f.write(csv_data)

# Save Excel
with open(f"{excel_name}.xlsx", "wb") as f:
    f.write(excel_buffer.getvalue())

print(f"File saved as {excel_name}.xlsx successfully!")

File saved as computations.xlsx successfully!
